In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_8477/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)


    def keep_random_models(self, n_models=3):
        # Extract block part from 'model' column
        self.results_df['block'] = self.results_df['model'].apply(lambda x: '_'.join(x.split('_')[1:3]))
        
        # Group by 'block' and keep n_models randomly from each group
        def select_random_models(group):
            return group.sample(n=min(n_models, len(group)), random_state=1)
        
        # Apply the function and ensure grouping columns are excluded from the final DataFrame
        grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)
        self.results_df = grouped.reset_index(drop=True).drop(columns=['block'])

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.keep_random_models()
        self.results_df = self.results_df.sort_values(by="r2_sup", ascending=False, ignore_index=True)
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.1)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



/tmp/ipykernel_8477/3323087823.py:36: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)


,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_2_37_2,0.737677,0.456346,0.745501,-0.438069,0.490084,3.635419,0.910330,0.608512,1.204950,0.700060,0.458429,0.711085,67.426356,139.181136,"Hidden Size=[8], regularizer=0.02, learning_ra..."
1,model_2_38_5,0.739324,0.443568,0.713364,0.826306,0.487006,3.720865,1.025280,0.429965,1.333438,0.697858,0.461831,0.708849,67.438957,139.193737,"Hidden Size=[8], regularizer=0.02, learning_ra..."
2,model_2_38_3,0.739971,0.443122,0.720290,0.855273,0.485798,3.723848,1.000507,0.358261,1.339129,0.696992,0.463166,0.707969,67.443925,139.198705,"Hidden Size=[8], regularizer=0.02, learning_ra..."
3,model_2_37_3,0.738144,0.443120,0.745943,-0.459862,0.489211,3.723858,0.908748,0.617734,1.210145,0.699436,0.459395,0.710451,67.429925,139.184705,"Hidden Size=[8], regularizer=0.02, learning_ra..."
4,model_2_11_4,0.746839,0.442482,0.549492,0.714604,0.472967,3.728124,1.611440,0.491969,1.309430,0.687726,0.477345,0.698557,67.497461,139.252241,"Hidden Size=[8], regularizer=0.02, learning_ra..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,model_3_4_9,0.778818,-10.259172,0.555666,0.220327,0.413223,75.290133,1.589357,1.008029,1.632133,0.642824,0.056288,0.652948,99.767535,206.312511,"Hidden Size=[12], regularizer=0.02, learning_r..."
71,model_3_6_0,0.803286,-11.321343,0.241214,0.802191,0.367510,82.392878,2.714133,0.619192,1.575433,0.606226,0.160688,0.615773,100.002012,206.546988,"Hidden Size=[12], regularizer=0.02, learning_r..."
72,model_3_5_9,0.770275,-12.329061,0.061415,-0.058877,0.429183,89.131489,3.357264,0.973181,1.777216,0.655121,0.019839,0.665438,99.691744,206.236720,"Hidden Size=[12], regularizer=0.02, learning_r..."
73,model_3_5_7,0.786808,-12.843796,0.122800,0.148277,0.398296,92.573530,3.137692,0.782792,1.783246,0.631107,0.090379,0.641046,99.841120,206.386096,"Hidden Size=[12], regularizer=0.02, learning_r..."
